<a href="https://colab.research.google.com/github/NizamuddinMandekar/ChatBot/blob/main/txt_to_ans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain-community==0.0.19

In [ ]:
pip install pypdf

In [ ]:
pip install langchain

In [ ]:
pip install langchain_google_genai

In [ ]:
pip install tiktoken

In [ ]:
pip install chromadb

In [ ]:
!pip install -q textract transformers openai faiss-cpu

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import tiktoken
os.environ["GOOGLE_API_KEY"]="YOUR_API_KEY"

In [9]:
llm=ChatGoogleGenerativeAI(model='gemini-pro',convert_system_message_to_human=True ,temperature=0)

In [10]:
folder_path = "/content/drive/MyDrive/txt_to_ans_pdfs"
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

print(pdf_files)

['cloud.pdf', 'contextai.pdf', 'web3.pdf']


In [11]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/txt_to_ans_pdfs/")
docs = loader.load()

In [12]:
def tiktoken_len(text):
        tokens = tiktoken.get_encoding('cl100k_base').encode(
            text,
            disallowed_special = ()
        )
        return len(tokens)

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
    length_function=tiktoken_len
)

In [14]:
documents=splitter.split_documents(docs)

In [15]:
len(documents)

13

In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embed = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [17]:
from langchain_community.vectorstores import Chroma

In [18]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

persist_dir='/content/drive/MyDrive/txt_to_ans_pdfs/test-db'
db = Chroma.from_documents(documents=documents, embedding=embed, persist_directory=persist_dir)



In [19]:
x=db.similarity_search('Context AI')

In [20]:
x[0]

Document(page_content='2. Digital Identity and Privacy: Web 3.0 addresses concerns related', metadata={'page': 1, 'source': '/content/drive/MyDrive/txt_to_ans_pdfs/web3.pdf'})

In [21]:
prompt="""You are a QnA Chatbot. Your task is to answer the question only based on the context given.
    Do not answer the question if the answer could not be generated from the context.
    Do not include unnecessary context in the answer.

    Note: Do not explicitly mention the word "context" in the answer.

    Context:
    {% for item in context %}
    source: {{item.metadata['source']}}
    content : {{item.page_content}}
    page: {{item.metadata['page']}}

    -----
    {% endfor %}
    question:{{question}}

    Answer:
    """

In [22]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(template=prompt,
                        input_variables=['question','context'],
                        template_format="jinja2"
                        )

In [23]:
retr=db.as_retriever()

In [24]:
from langchain.schema.runnable import RunnableMap,RunnableLambda, RunnablePassthrough,RunnableParallel
from langchain.schema.output_parser import StrOutputParser

In [32]:
chain = RunnableParallel({
    "context": retr,
    "question":RunnablePassthrough()
    }) | prompt | llm | StrOutputParser()

answer=chain.invoke('What is Natural Language Processing(NLP)?')


In [27]:
from IPython.display import display
import ipywidgets as widgets
from langchain.chains import ConversationalRetrievalChain

# Create conversation chain that uses our vectordb as retriver, this also allows for chat history management
qa = ConversationalRetrievalChain.from_llm(ChatGoogleGenerativeAI(model='gemini-pro',convert_system_message_to_human=True, temperature=0.1), db.as_retriever())


In [45]:
chat_history = []

def on_submit(_):
    query = input_box.value
    input_box.value = ""

    if query.lower() == 'exit':
        print("Thank You!")
        return

    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result['answer']))

    display(widgets.HTML(f'<b>User:</b> {query}'))
    display(widgets.HTML(f'<b><font color="blue">Chatbot:</font></b> {result["answer"]}'))

print("Welcome to the Chatbot! Type 'exit' to stop.")

input_box = widgets.Text(placeholder='Please enter your question:')
input_box.on_submit(on_submit)

display(input_box)

Welcome to the Chatbot! Type 'exit' to stop.


Text(value='', placeholder='Please enter your question:')

HTML(value='<b>User:</b> What is Natural Language Processing(NLP)?')

HTML(value='<b><font color="blue">Chatbot:</font></b> Natural Language Processing (NLP) is a key component of …